#### Importiamo le librerie necessarie. Useremo ResNet50.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications import resnet50, inception_v3
import pandas as pd
import datetime, os
import shutil
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Load the TensorBoard notebook extension
%load_ext tensorboard

keras.__version__

'2.4.0'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import zipfile, os
# this would take very long !!!
base_dir = '/content/drive/MyDrive/DSIM_project/images_animals10_split.zip'
zip = zipfile.ZipFile(f'{base_dir}')
zip.extractall()
zip.close()

## Creiamo train e test set

In [6]:
dest_train = "images_animals10_split/train/"
dest_test = "images_animals10_split/test/"
dest_val = "images_animals10_split/val/"

## Prima configurazione

#### Creiamo i generators

In [7]:
train_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= resnet50.preprocess_input,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                brightness_range=[0.5, 1.5])


train_generator = train_processing.flow_from_directory(directory = dest_train,
                                    target_size = (224, 224),
                                    color_mode = "rgb",
                                    batch_size = 32,
                                    class_mode = "categorical",
                                    shuffle = True,
                                    seed = 1234)

Found 7360 images belonging to 10 classes.


In [8]:
val_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

val_generator = val_processing.flow_from_directory(
        directory=dest_val,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 916 images belonging to 10 classes.


In [9]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

test_generator = test_processing.flow_from_directory(
        directory=dest_test,
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

Found 927 images belonging to 10 classes.


#### Importiamo il modello con i pesi adatti (imagenet), e settiamo i layer iniziali come non allenabili

In [10]:
base_net = resnet50.ResNet50(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3), pooling="avg")

94773248/94765736 [==============================] - 1s 0us/step


In [11]:
for layer in base_net.layers:
    layer.trainable = False

#### Ora aggiungiamo due fully connected layers e poi creiamo un nuovo modello "net", dato dalla combinazione dei due 

In [13]:
# Output of the base_net model
x = base_net.output
# intermediate fully-connected layer + ReLU
x = keras.layers.Dense(1024, activation='relu')(x)
# final fully-connected layer + SoftMax 
pred = keras.layers.Dense(10, activation='softmax')(x)

In [14]:
net1 = keras.Model(inputs=base_net.input, outputs=pred)

In [15]:
net1.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

#### Compiliamo il modello usando come loss la categorical crossentropy, come optimizer RMSprop e come metrica l'accuracy

In [16]:
net1.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.RMSprop(),
            metrics=['accuracy'])

#### Ora fittiamo il modello con 10 epoche e visualizziamo i risultati in termini di accuracy e loss su train e test set

In [19]:
cp_callback = keras.callbacks.ModelCheckpoint(filepath="bidim_weights",
                                              save_best_only=True,
                                                 save_weights_only=True,
                                                 monitor="val_accuracy",
                                                 verbose=1)


In [20]:
history1 = net1.fit(train_generator,
          epochs=1,
          validation_data=val_generator,
          callbacks=[cp_callback])

  1/230 [..............................] - ETA: 0s - loss: 2.6409 - accuracy: 0.1250WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
230/230 [==============================] - ETA: 0s - loss: 0.9163 - accuracy: 0.8346
Epoch 00001: saving model to bidim_weights.ckpt
230/230 [==============================] - 219s 953ms/step - loss: 0.9163 - accuracy: 0.8346 - val_loss: 0.2815 - val_accuracy: 0.9236


In [ ]:
#Open tensorboard
#Open localhost:6006

In [21]:
def accuracy_loss(history):
    plt.figure(figsize=(20,10))
    #
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label = "train_accuracy", color = "red")
    plt.plot(history.history['val_accuracy'], label="train_accuracy", color ="blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Accuracy', size = 15)
    #
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label = "Train loss", color = "red")
    plt.plot(history.history['val_loss'], label="Test loss", color = "blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Loss', size = 15)

In [31]:
bidim_model = net1.to_json()

with open("bidim_model.json", "w") as json_file:
    json_file.write(bidim_model)

In [27]:
load_status = net1.load_weights("bidim_weights")
load_status.assert_consumed()

In [32]:
performance1 = net1.evaluate(test_generator)

29/29 [==============================] - 16s 567ms/step - loss: 0.2073 - accuracy: 0.9320


In [ ]:
accuracy_loss(history1)